In [1]:
import os
import copy
import tempfile
import json
from subprocess import call
from diConstants import (HG19_ALL_CHROMS, MM9_ALL_CHROMS,
    HG19_TRAIN_CHROMS, MM9_TRAIN_CHROMS,
    VALID_CHROMS, TEST_CHROMS) 

import models
import modelTemplates

Using Theano backend.


# Model information
### Dependency
* Python == 2.x
* Keras == 1.x

### Marks
* ['H3K27AC', 'H3K4ME1', 'H3K4ME3', 'H3K27ME3', 'H3K36ME3'] 총 5개가 있고 추가로 'INPUT'이라는 문자열이 들어감
* 데이터셋의 이름인 [GM12878_5+1marks-K4me3_all]에서 5+1marks가 이를 뜻한다

### How many models exist?
* 현재 [GM12878_5+1marks-K4me3_all]데이터셋으로 부터 5개의 mark에 대한 값들로 학습을 진행한다
* 출력은 위의 5개 마크들 중 1개로 구성된다. 즉 서로다른 5개의 모델이 생성된다
* 추가로 Peak를 classification하느냐, Signal을 regression하느냐에 따라 각 mark마다 2개의 모델이 구성된다

### Signal regression ver. model
* Mean Square Error 오차함수를 사용한다
* Relu함수를 활성화함수로 사용한다

### Peak classification ver. model
* Binary Classification Error 오차함수를 사용한다
* Sigmoid함수를 활성화함수로 사용한다

### Input & Output
* 5개의 mark데이터를 학습해서 1개의 mark에 대한 작업을 수행
* 각 mark에는 위에 나와있듯이 2개의 모델이 존재

### Normalization
* 학습에 사용되는 데이터셋은 normalization(표준화 작업)을 수행한다
* arcsinh함수를 사용(Inverse of hyperbolic sin function)

### In this code..
* load data and init Dataset class
* initialize Keras prediction models and save their structual(CNN-layer, Activation, Error, etc...) as JSON format

In [2]:
GM_MARKS = ['H3K27AC'] # Is this output mark?
#GM_MARKS = ['H3K27AC', 'H3K4ME1', 'H3K4ME3', 'H3K27ME3', 'H3K36ME3']
def test_GM18526():
    for test_cell_line in ['GM18526']:
        for subsample_target_string in ['0.5e6']:
            for predict_binary_output in [True, False]: # original : TRUE , FALSE   
                for output_mark in GM_MARKS:                            

                    model_params = modelTemplates.make_model_params(
                        model_library='keras',
                        model_class='SeqToPoint',
                        model_type='dcgan',
                        model_specific_params={
                            'num_filters': 6,
                            'filter_length': 51
                        },
                        compile_params={            
                            'optimizer': 'adagrad'
                        },
                        dataset_params={
                            'train_dataset_name': 'GM12878_5+1marks-K4me3_all',
                            'test_dataset_name': '%s_5+1marks-K4me3_all' % test_cell_line, 
                            'num_train_examples': 1000,
                            'seq_length': 1001,
                            'peak_fraction': 0.5,                            
                            'train_X_subsample_target_string': subsample_target_string,
                            'num_bins_to_test': None,
                            'train_chroms': HG19_TRAIN_CHROMS,
                            'test_chroms': TEST_CHROMS,
                            'only_chr1': True
                        },
                        output_marks=[output_mark],
                        train_params={
                            'nb_epoch': 1,
                            'batch_size': 100
                        },
                        predict_binary_output=predict_binary_output,
                        zero_out_non_bins=True,
                        generate_bigWig=True)

                    init_model(model_params)

def init_model(model_params):
    m = models.SeqModel.instantiate_model(model_params)
    print '\n\n\n'

In [3]:
test_GM18526()

Generator model summary
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, None, 6)       1842        convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, 6)       0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, None, 1)       6007        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, None, 1)       0           convolution1d_2[0][0]            
Total params: 7,849
Trainable params: 7,849
Non-trainable params: 0